In [1]:
import rHeston
import BlackScholes as bs
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import ImpliedDrift as iD
import time

from matplotlib import cm

In [2]:
# Read the dataset
dates = np.array(["23_01_23.csv", "24_01_23.csv", "25_01_23.csv", "26_01_23.csv", "27_01_23.csv",
                  "30_01_23.csv", "06_02_23.csv", "13_02_23.csv", "21_02_23.csv"])

spot = np.array(pd.read_csv("spot.csv").Spot).flatten()
moneyness = np.array([80.0,90.0,95.0,97.5,100.0,102.5,105.0,110.0,120.0])

index = 0
data = pd.read_csv(dates[index]); S0 = spot[index];
K = S0 * moneyness / 100

if index:
    maturities = np.array(data.Tenor).flatten()
    IV = np.array(data.drop(columns = ['Date', 'Tenor']))/100.

else:
    maturities = np.array(data['Exp Date']).flatten()
    IV = np.array(data.drop(columns = 'Exp Date'))/100.
    

maturities = maturities[maturities*365 <= 180]
    
nr = len(maturities); nc = len(K);

IV = IV[:nr]

param = np.array(pd.read_csv("rHeston_data.csv"))

In [3]:
r = iD.r(maturities, index); q = iD.q(maturities, index);
options_type = np.ones(nc)

def f(prms):
    rho, theta, H = prms
    iv = np.zeros([nr, nc])   
    
    for i in range(nr):
        iv[i] = rHeston.analytic_rhest(S0, K, maturities[i], H, rho, theta, options_type)
        iv[i] = bs.BSImpliedVol(S0, K, maturities[i], r[i], q[i], iv[i], Option_type = 1, toll = 1e-5)
    
    return np.mean(abs(IV-iv)/IV)*100, iv

In [4]:
%%timeit
f(param[0])

1.07 s ± 31.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
t = time.time()
iv_list = list()
length = len(param); err = np.zeros(length);

for i in range(length):
    err[i], aux = f(param[i])
    iv_list.append(aux)
    if (i+1)%100 == 0:
        print(f"Iteration progress: {(i+1)/length*100: .2f}%")
    
print(f"\nElapsed time: {time.time()-t: .2f}s")

Iteration progress:  1.25%
Iteration progress:  2.50%
Iteration progress:  3.75%
Iteration progress:  5.00%
Iteration progress:  6.25%
Iteration progress:  7.50%
Iteration progress:  8.75%
Iteration progress:  10.00%
Iteration progress:  11.25%
Iteration progress:  12.50%
Iteration progress:  13.75%
Iteration progress:  15.00%
Iteration progress:  16.25%
Iteration progress:  17.50%
Iteration progress:  18.75%
Iteration progress:  20.00%
Iteration progress:  21.25%
Iteration progress:  22.50%
Iteration progress:  23.75%
Iteration progress:  25.00%
Iteration progress:  26.25%
Iteration progress:  27.50%
Iteration progress:  28.75%
Iteration progress:  30.00%
Iteration progress:  31.25%
Iteration progress:  32.50%
Iteration progress:  33.75%
Iteration progress:  35.00%
Iteration progress:  36.25%
Iteration progress:  37.50%
Iteration progress:  38.75%
Iteration progress:  40.00%
Iteration progress:  41.25%
Iteration progress:  42.50%
Iteration progress:  43.75%
Iteration progress:  45.00%

In [6]:
err.min()

1.9191016120020916

In [8]:
err.max()

18.255056533653956

In [9]:
err.mean()

3.6416046368506314

In [10]:
err.std()

1.6019668075005524

In [11]:
err.var()

2.5662976523335117

In [12]:
param[np.argmin(err)]

array([-0.69375051,  0.49382334,  0.27394262])

In [13]:
df = pd.DataFrame(err, columns = ["Error"])
df.to_csv("grid_error.csv", index = False)

In [14]:
iv = np.zeros([length, len(iv_list[0].flatten())])

for i in range(length):
    iv[i] = iv_list[i].flatten()

In [15]:
db = pd.DataFrame(iv)
db.to_csv("grid_iv.csv", index = False)